# AirBNB tracker

In [1]:
import numpy as np
import datetime
import pandas as pd
import pyperclip
import pickle
import re

In [2]:
tous_les_logements = pickle.load(open('tous_les_logements.p', 'rb'))
print(tous_les_logements.shape)
tous_les_logements.tail(2)

(214, 12)


,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
301,2023-12-25,Hébergement ⋅ Cachan\nBambouseraie aux portes ...,Hébergement,Cachan,Bambouseraie aux portes de PARIS !,963,4.74,19.0,None,None,False,False
302,2023-12-25,Hébergement ⋅ Arcueil\nMaison élégante près de...,Hébergement,Arcueil,Maison élégante près de Paris,447,NaN,NaN,None,None,False,False


In [79]:
class Logement:
    __slots__ = [
        'date_enregistrement',
        'texte',
        'type',
        'ville',
        'description',
        'prix',
        'note',
        'nb_avis',
        'nb_lits',
        'nb_chambres',
        'est_professionnel',
        'est_nouveau'
    ]
    def __init__(self, ville: str, type: str, description: str,
                 nb_lits: int, nb_chambres: int,
                 est_professionnel: bool, est_nouveau: bool,
                 prix: int, note: float, nb_avis: int,
                 texte: str):
        self.type = type
        self.ville = ville
        self.description = description
        self.nb_lits = nb_lits
        self.nb_chambres = nb_chambres
        self.prix = prix
        self.est_nouveau = est_nouveau
        self.est_professionnel = est_professionnel
        self.note = note
        self.nb_avis = nb_avis

        self.date_enregistrement = datetime.date.today()
        self.texte = texte
    
    @classmethod
    def from_text(cls, texte):
        texte = texte.strip()
        attrs = set(cls.__slots__) - {'texte', 'date_enregistrement'}
        kwargs = dict(texte=texte)
        for attr in attrs:
            kwargs[attr] = eval(f'cls._get_{attr}')(texte)
        return cls(**kwargs)
    
    
    @staticmethod
    def _get_type(texte: str):
        pat = ' (⋅|·) '
        return re.split(pat, texte.split('\n')[0])[0]
        #return texte.split('\n')[0].split(pat)[0]
    
    @staticmethod
    def _get_ville(texte: str):
        pat = ' (⋅|·) '
        return re.split(pat, texte.split('\n')[0])[2]
        #return texte.split('\n')[0].split(' ⋅ ')[1]

    @staticmethod
    def _get_description(text: str):
        return text.split('\n')[1]
    
    @staticmethod
    def _get_nb_lits(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[0]

    @staticmethod
    def _get_nb_chambres(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[1]
    
    @staticmethod
    def _get_nb_lits_nb_chambres(texte: str) -> tuple:
        nb_lits, nb_chambres = None, None
        pat = r'(?P<nb_lits>\d) lits?'
        if (rgx := re.search(pat, texte.split('\n')[2])):
            nb_lits = int(rgx.group()[0])
        pat = r'(?P<nb_chambres>\d) chambres?'
        if (rgx := re.search(pat, texte.split('\n')[2])):
            nb_chambres = int(rgx.group()[0])
        return nb_lits, nb_chambres
    
    @staticmethod
    def _get_est_professionnel(texte: str):
        return texte.split('\n')[3] == 'Professionnel'
    
    @staticmethod
    def _get_prix(texte: str):
        for ligne in texte.split('\n'):
            if rgx := re.match('(?P<prix>[\d\u202f]+) € par nuit', ligne):
                return int(rgx.groups()[0].replace('\u202f', ''))
        raise ValueError('Pas de prix trouvé sur :\n' + texte)
    
    @staticmethod
    def _get_est_nouveau(texte: str):
        return texte.split('\n')[-1] == 'Nouveau'
    
    @staticmethod
    def _get_note(texte: str):
        return Logement._get_note_et_nb_avis(texte)[0]

    @staticmethod
    def _get_nb_avis(texte: str):
        return Logement._get_note_et_nb_avis(texte)[1]

    @staticmethod
    def _get_note_et_nb_avis(texte: str):
        note, nb_avis = None, None
        pat = r'(?P<note>\d\,\d\d?) \((?P<nb_avis>\d+)\)'
        if rgx := re.search(pat, texte):
            note = float(rgx.groupdict()['note'].replace(',', '.'))
            nb_avis = int(rgx.groupdict()['nb_avis'])
        return note, nb_avis
    
    def __repr__(self):
        rep = f'{self.__class__.__name__}('
        for attr in ('type', 'ville', 'prix'):
            rep += f'\n    {attr}={getattr(self, attr)},'
        rep = rep[:-1] + '\n)'
        return rep


def tronque_à_gauche(texte: str) -> str:
    """Tronque à gauche le texte avant 'Classement des résultats'"""
    stop = texte.find('Classement des résultats')
    return texte[stop + len('Classement des résultats'):].strip()

def tronque_à_droite(texte: str) -> str:
    """Tronque à droite le texte qui dépasse de la numérotation des pages"""
    # stop = texte.find('1\n2\n3\n4')
    stop = re.search(r'[\d…]\n[\d…]\n[\d…]', texte)
    if stop:
        return texte[:stop.start()].strip()
    stop = re.search(r'Carte Google \d+ séjours affichés.', texte)
    texte = texte[:stop.start()].strip()
    stop = re.search(r'Un voyage, deux logements', texte)
    if stop:
        texte = texte[:stop.start()].strip()
    return texte

In [89]:
page = pyperclip.paste()

page = tronque_à_gauche(page)
page = tronque_à_droite(page)

In [90]:
pat = '(Appartement|Chambre|Hébergement|Tiny house|Appartement en résidence|Villa|Maison de ville) [⋅·] '
splits = [
    match.start()
    for match in re.finditer(pat, page)
]
splits += [len(page)]

logements = []
for texte in [page[start:stop] for start, stop in zip(splits, splits[1:])]:
    logements.append(Logement.from_text(texte))

In [91]:
dico = [{attr: getattr(logement, attr) for attr in Logement.__slots__} for logement in logements]
page_df = pd.DataFrame(dico, index=range(len(dico)))

In [93]:
assert page_df.shape[0] == 18
assert page_df.drop(['nb_lits', 'nb_chambres'], axis=1).notnull().sum().min() > 0

In [94]:
# On ajoute à tous les logements la page en cours de scrapping : 
tous_les_logements = pd.concat([tous_les_logements, page_df], axis=0, ignore_index=True)
print(tous_les_logements.shape)
tous_les_logements.tail()

(393, 12)


,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
388,2024-01-06,Hébergement ⋅ Cachan\nCharmante maison françai...,Hébergement,Cachan,Charmante maison française familiale avec jard...,586,4.87,15.0,2,2,False,False
389,2024-01-06,Appartement ⋅ Cachan\nAppartement très calme.\...,Appartement,Cachan,Appartement très calme.,449,NaN,NaN,2,2,False,True
390,2024-01-06,Appartement ⋅ Cachan\nJoli 3 pièces à 10 min d...,Appartement,Cachan,Joli 3 pièces à 10 min de Paris,476,NaN,NaN,2,2,False,False
391,2024-01-06,Appartement ⋅ Cachan\nappartement t1 indépenda...,Appartement,Cachan,appartement t1 indépendant. (ga),195,NaN,NaN,2,1,False,True
392,2024-01-06,Appartement ⋅ Cachan\n2P a 10 min à pied du RE...,Appartement,Cachan,2P a 10 min à pied du RER B,395,NaN,NaN,2,1,False,False


Quand on a fini d'itérer sur les pages, on dump :

In [95]:
pickle.dump(tous_les_logements.drop_duplicates().fillna(np.nan), open('tous_les_logements.p', 'wb'))